In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import statsmodels
import scipy
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import scipy.stats
import pylab
import matplotlib as mpl
from scipy.stats.distributions import chi2
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import norm
from arch import arch_model
import yfinance
import datetime
sns.set()

In [2]:
raw_data=yfinance.download(tickers="^GSPC ^FTSE ^N225 ^GDAXI",start='1994-01-07',
                           end="2019-09-01",interval='1d',group_by='ticker',auto_adjust=True,threads=True
                          )
df_comp=raw_data.copy()

[*********************100%***********************]  4 of 4 completed


In [3]:
df_comp['nikkei']=df_comp['^N225'].Close[:]
df_comp['ftse']=df_comp['^FTSE'].Close[:]
df_comp['spx']=df_comp['^GSPC'].Close[:]
df_comp['dax']=df_comp['^GDAXI'].Close[:]
df_comp.drop(columns=['^N225','^FTSE','^GSPC','^GDAXI'],inplace=True)

C:\Users\adadi\anaconda3\envs\DataScienceEnv\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [4]:
df_comp.head(3)

Ticker,nikkei,ftse,spx,dax
Price,,,,
Date,,,,
1994-01-07 00:00:00+00:00,18124.009766,3446.000000,469.899994,2224.949951
1994-01-10 00:00:00+00:00,18443.439453,3440.600098,475.269989,2225.000000
1994-01-11 00:00:00+00:00,18485.250000,3413.800049,474.130005,2228.100098


In [5]:
df_comp['ret_nikkei']=df_comp['nikkei'].pct_change(1).mul(100)
df_comp['ret_ftse']=df_comp['ftse'].pct_change(1).mul(100)
df_comp['ret_spx']=df_comp['spx'].pct_change(1).mul(100)
df_comp['ret_dax']=df_comp['dax'].pct_change(1).mul(100)

In [6]:
df_comp['norm_ret_nikkei']=df_comp['ret_nikkei'].div(df_comp['ret_nikkei'][1])*100
df_comp['norm_ret_ftse']=df_comp['ret_ftse'].div(df_comp['ret_ftse'][1])*100
df_comp['norm_ret_spx']=df_comp['ret_spx'].div(df_comp['ret_spx'][1])*100
df_comp['norm_ret_dax']=df_comp['ret_dax'].div(df_comp['ret_dax'][1])*100

In [7]:
df_comp.isin([np.inf, -np.inf]).sum()

Ticker           Price
nikkei                    0
ftse                      0
spx                       0
dax                       0
ret_nikkei                0
ret_ftse                  0
ret_spx                   0
ret_dax                   0
norm_ret_nikkei           0
norm_ret_ftse             0
norm_ret_spx              0
norm_ret_dax              0
dtype: int64

In [8]:
df_comp.index=df_comp.index.date

In [9]:
df_comp.shape

(6673, 12)

In [10]:
df_comp=df_comp.asfreq('b')

In [11]:
df_comp.isna().sum()

Ticker           Price
nikkei                    390
ftse                      211
spx                       233
dax                       197
ret_nikkei                 19
ret_ftse                   19
ret_spx                    19
ret_dax                    19
norm_ret_nikkei            19
norm_ret_ftse              19
norm_ret_spx               19
norm_ret_dax               19
dtype: int64

In [12]:
df_comp.ftse.fillna(method='ffill',inplace=True)
df_comp.spx.fillna(method='ffill',inplace=True)
df_comp.nikkei.fillna(method='ffill',inplace=True)
df_comp.dax.fillna(method='ffill',inplace=True)
df_comp.ret_ftse.fillna(method='ffill',inplace=True)
df_comp.ret_spx.fillna(method='ffill',inplace=True)
df_comp.ret_nikkei.fillna(method='ffill',inplace=True)
df_comp.ret_dax.fillna(method='ffill',inplace=True)
df_comp.norm_ret_ftse.fillna(method='ffill',inplace=True)
df_comp.norm_ret_spx.fillna(method='ffill',inplace=True)
df_comp.norm_ret_nikkei.fillna(method='ffill',inplace=True)
df_comp.norm_ret_dax.fillna(method='ffill',inplace=True)

C:\Users\adadi\anaconda3\envs\DataScienceEnv\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [13]:
df_comp.ftse.isna().sum(),df_comp.spx.isna().sum(),df_comp.nikkei.isna().sum(),df_comp.dax.isna().sum()

(0, 0, 0, 0)

In [14]:
df_comp.shape

(6691, 12)

In [15]:
size=int(len(df_comp)*0.8)
df,df_test = df_comp.iloc[:size],df_comp.iloc[size:]

In [16]:
df.index.max(),df.index.min(),df_test.index.max(),df_test.index.min()

(Timestamp('2014-07-14 00:00:00', freq='B'),
 Timestamp('1994-01-07 00:00:00', freq='B'),
 Timestamp('2019-08-30 00:00:00', freq='B'),
 Timestamp('2014-07-15 00:00:00', freq='B'))